In [4]:
from query_rewrtiting import generate_rewritten_queries, model, tokenizer
from rag import RAG
from reranking import rerank_documents

def compare_retrieval_methods(query: str, k: int = 5):
    rag = RAG()
    
    print("=== Original Query ===")
    print(f"Query: {query}\n")
    
    # Normal querying results
    print("=== Results from Normal Querying ===")
    normal_results = rag.retrive_relevent_docs(query, top_k=k)
    for i, hit in enumerate(normal_results[0], 1):
        metadata = hit.entity.get('metadata')
        print(f"\n{i}. Luật: {metadata.get('luật')}")
        print(f"Điều {metadata.get('điều')}: {metadata.get('tên_điều')}")
        print(f"Nội dung: {metadata.get('chunk_content')}")
    
    # Query rewriting + reranking results
    print("\n=== Results from Query Rewriting + Reranking ===")
    rewritten_queries = generate_rewritten_queries(query, model, tokenizer)
    print("\nRewritten Queries:")
    for i, q in enumerate(rewritten_queries, 1):
        print(f"{i}. {q}")
    
    # Get results for each rewritten query
    all_results = []
    for q in rewritten_queries:
        results = rag.retrive_relevent_docs(q, top_k=k)
        all_results.extend(results[0])
    
    # Rerank results
    reranked_results = rerank_documents(query, all_results, top_k=k)
    print("\nReranked Results:")
    for i, (doc, score) in enumerate(reranked_results, 1):
        print(f"\n{i}. Relevance Score: {score}")
        print(doc)

if __name__ == "__main__":
    test_query = "Hình phạt cho tội trộm cắp tài sản là gì?"
    compare_retrieval_methods(test_query, k=10)

Device set to use mps:0


=== Original Query ===
Query: Hình phạt cho tội trộm cắp tài sản là gì?

=== Results from Normal Querying ===

1. Luật: BỘ LUẬT HÌNH SỰ
Điều Điều 300: Tội tài trợ khủng bố
Nội dung: 1. Người nào huy động, hỗ trợ tiền, tài sản dưới bất kỳ hình thức nào cho tổ chức, cá nhân khủng bố, thì bị phạt tù từ 05 năm đến 10 năm.
2. Người chuẩn bị phạm tội này, thì bị phạt tù từ 01 năm đến 05 năm.
3. Người phạm tội còn có thể bị phạt quản chế, cấm cư trú từ 01 năm đến 05 năm hoặc tịch thu một phần hoặc toàn bộ tài sản.

2. Luật: BỘ LUẬT HÌNH SỰ
Điều Điều 339: Tội giả mạo chức vụ, cấp bậc, vị trí công tác
Nội dung: Người nào giả mạo chức vụ, cấp bậc, vị trí công tác thực hiện hành vi trái pháp luật nhưng không nhằm mục đích chiếm đoạt tài sản, thì bị phạt cải tạo không giam giữ đến 02 năm hoặc phạt tù từ 03 tháng đến 02 năm.

3. Luật: LUẬT TRƯNG MUA, TRƯNG DỤNG TÀI SẢN
Điều Điều 34: Bồi thường thiệt hại do việc trưng dụng tài sản gây ra
Nội dung: 4. Trường hợp tài sản trưng dụng là tài sản nhà nước

In [7]:
import nltk
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def evaluate_bleu(reference, candidate):
    reference_tokens = [ref.split() for ref in reference]
    candidate_tokens = candidate.split()
    smoothie = SmoothingFunction().method1
    
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothie)
    return bleu_score

def evaluate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score("\n".join(reference), candidate)
    
    return {key: scores[key].fmeasure for key in scores}

if __name__ == "__main__":
    GPT_sentences = [
        "1. Người phạm tội còn có thể bị phạt quản chế, cấm cư trú từ 01 năm đến 05 năm.",
        """
            1. Cơ quan có thẩm quyền trong bảo vệ an ninh mạng; giúp đỡ, tạo điều kiện cho cơ quan, tổ chức và người có trách nhiệm tiến hành các biện pháp bải bảo vảo vệ an ninhệ an ninh;

            2.
            3. Cười đi đi đi đi luềuềuềuềuề
        """
    ]
    Llama_sentences = [
        "Hình phạt cho tội trộm cắp tài sản khi lợi dụng hoàn cảnh chiến tranh, tình trạng khẩn cấp là bị phạt tiền từ 10.000.000 đồng đến 100.000.000 đồng hoặc tịch thu một phần hoặc toàn bộ tài sản.",
        """
            Luật An Ninh Mạng là một bộ luật pháp quan trọng trong lĩnh vực an ninh mạng của Việt Nam. Bộ luật này quy định các nguyên tắc, quy định và trách nhiệm của cơ quan, tổ chức và cá nhân liên quan đến an ninh mạng.

            Dưới đây là một số điều chính của Luật An Ninh Mạng:

            1. **Giải thích từ ngữ**: Bộ luật định nghĩa từ "an ninh mạng" và các khái niệm liên quan khác.
            2. **Tình huống nguy hiểm về an ninh mạng**: Bộ luật quy định rằng tình huống nguy hiểm về an ninh mạng là sự việc xảy ra trên không gian mạng khi có hành vi xâm phạm nghiêm trọng an ninh quốc gia, gây tổn hại đặc biệt nghiêm trọng trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân.
            3. **Trách nhiệm của cơ quan, tổ chức, cá nhân sử dụng không gian mạng**: Bộ luật quy định các trách nhiệm của cơ quan, tổ chức và cá nhân liên quan đến việc sử dụng không gian mạng, bao gồm:
            - Tuân thủ quy định của pháp luật về an ninh mạng.
            - Cung cấp thông tin liên quan đến bảo vệ an ninh mạng cho cơ quan có thẩm quyền.
            - Thực hiện yêu cầu và hướng dẫn của cơ quan có thẩm quyền trong bảo vệ an ninh mạng.
            - Giúp đỡ, tạo điều kiện cho cơ quan, tổ chức và người có trách nhiệm tiến hành các biện pháp bảo vệ an ninh mạng.
            4. **Các hành vi bị nghiêm cấm về an ninh mạng**: Bộ luật quy định các hành vi bị nghiêm cấm về an ninh mạng, bao gồm:
            - Chống lại hoặc cản trở hoạt động của lực lượng bảo vệ an ninh mạng.
            - Tấn công, vô hiệu hóa trái pháp luật làm mất tác dụng biện pháp bảo vệ an ninh mạng.
            - Lợi dụng hoặc lạm dụng hoạt động bảo vệ an ninh mạng để xâm phạm chủ quyền, lợi ích, an ninh quốc gia, trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân hoặc để trục lợi.
            - Hành vi khác vi phạm quy định của Luật này.

            Tóm lại, Luật An Ninh Mạng là một bộ luật pháp quan trọng trong lĩnh vực an ninh mạng của Việt Nam, quy định các nguyên tắc, quy định và trách nhiệm của cơ quan, tổ chức và cá nhân liên quan đến an ninh mạng.
        """
    ]
    candidate_sentences = [
        "Tại Điểm a khoản 1 Điều 15 Nghị định 167/2013/NĐ-CP quy định: Tội trộm cắp tài sản bị phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng. Bên cạnh đó, tùy thuộc vào thiệt hại về tài sản và mức độ nghiêm trọng gây ra, người có hành vi trộm cắp tài sản còn có thể bị truy cứu trách nhiệm hình sự trước pháp luật.",
        "An ninh mạng là sự bảo đảm hoạt động trên không gian mạng không gây phương hại đến an ninh quốc gia, trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân."
    ]
    
    results = []
    for model_name, references in zip(["Llama", "GPT"], [Llama_sentences, GPT_sentences]):
        for ref, candidate in zip(references, candidate_sentences):
            bleu = evaluate_bleu([ref], candidate)
            rouge = evaluate_rouge([ref], candidate)
            results.append({"Model": model_name, "Candidate": candidate, "BLEU Score": bleu, "ROUGE-1": rouge['rouge1'], "ROUGE-2": rouge['rouge2'], "ROUGE-L": rouge['rougeL']})
    
    df = pd.DataFrame(results)
    print(df)
    
    avg_scores = df.groupby("Model")[["BLEU Score", "ROUGE-1", "ROUGE-2", "ROUGE-L"]].mean().reset_index()
    print("\nAverage Scores:")
    print(avg_scores)

   Model                                          Candidate  BLEU Score  \
0  Llama  Tại Điểm a khoản 1 Điều 15 Nghị định 167/2013/...    0.078467   
1  Llama  An ninh mạng là sự bảo đảm hoạt động trên khôn...    0.000051   
2    GPT  Tại Điểm a khoản 1 Điều 15 Nghị định 167/2013/...    0.043086   
3    GPT  An ninh mạng là sự bảo đảm hoạt động trên khôn...    0.028100   

    ROUGE-1   ROUGE-2   ROUGE-L  
0  0.610169  0.331429  0.384181  
1  0.174157  0.149296  0.168539  
2  0.379562  0.148148  0.262774  
3  0.552239  0.181818  0.238806  

Average Scores:
   Model  BLEU Score   ROUGE-1   ROUGE-2  ROUGE-L
0    GPT    0.035593  0.465900  0.164983  0.25079
1  Llama    0.039259  0.392163  0.240362  0.27636
